<a href="https://colab.research.google.com/github/zzephryyk/ApiTelegram/blob/main/Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests bs4 img2pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.8 MB/s eta 0:00:00
  Created wheel for img2pdf: filename=img2pdf-0.5.1-py3-none-any.whl size=49275 sha256=92e09b4bd7d58ec8388598ccc8be94e627afab1710f5482875133047214c7f84
  Stored in directory: /root/.cache/pip/wheels/b7/e2/32/e12ea533072415219fc91a248cc413f34c505881ba850bf058
Successfully built img2pdf


In [2]:
import io
import img2pdf
from PIL import Image
import os
from typing import List


class mangaPdf:
    def __init__(
        self,
        ChapterName: str,
        imagesBytes: List[bytes],
        pdfPath: str,
    ) -> None:
        self.ChapterName = ChapterName
        self.imagesBytes = imagesBytes
        self.pdfPath = pdfPath

    def remove_alpha_channel(self, image_bytes: bytes) -> bytes:
        with Image.open(io.BytesIO(image_bytes)) as img:
            if img.mode == "RGBA":
                img = img.convert("RGB")
                with io.BytesIO() as buffer:
                    img.save(buffer, format="PNG")
                    return buffer.getvalue()
            return image_bytes

    def toPdf(self) -> None:
        if not os.path.exists(self.pdfPath):
            os.mkdir(self.pdfPath)
        pdf_file_path = os.path.join(self.pdfPath, f"{self.ChapterName}.pdf")
        if os.path.exists(pdf_file_path):
            return

        image_bytes_list = []
        for image_bytes in self.imagesBytes:
            processed_image_bytes = self.remove_alpha_channel(image_bytes)
            image_bytes_list.append(io.BytesIO(processed_image_bytes))

        if image_bytes_list:
            pdf_data = img2pdf.convert(
                [image_bytes_io.getvalue() for image_bytes_io in image_bytes_list],
                rotation=img2pdf.Rotation.ifvalid,
            )

            with open(pdf_file_path, "wb") as file:
                file.write(pdf_data)
            print(f"PDF saved to {pdf_file_path}")
        else:
            print(f"No images were processed. {self.imagesBytes}")

In [3]:
import json
import requests
from bs4 import BeautifulSoup

def configure_json(f):
    newJson = {}
    data = json.loads(f)
    chapSelected = data["actual"]
    chapter = next((i for i in data["chapters"] if str(i["chapter_number"]) == chapSelected), None)
    if chapter is not None:
        chapter_name = chapter["chapter_name"]
        newJson["site_url"] = data["site_url"]
        newJson["Karte"] = f"Karte {chapSelected}: {chapter_name}"
        newJson["title"] = data["title"]
        newJson["images"] = [
            data["image_url"] + "/"+ data["title"] + "_"+ image["manga_id"] + "/" "ch_"+ data["actual"] + "/" + image["image_name"]
            for image in data["images"]
        ]
        newJson["url"] = data["image_url"] + data["title"]
        newJson["chapter"] = chapSelected
        return newJson
    return {}

def ensureWeb(url):
    r1 = requests.get(url)
    if not r1.ok:
        raise Exception("URL Invalid or Blocked")
    soup = BeautifulSoup(r1.text, "html.parser")
    return soup.find("script", {"id": "suck-mihon-js-extra"}).text.split("= ")[1].split(""";
/* ]]> */\n""")[0]

url = str(input("URL: "))
f = ensureWeb(url)
jsonConfigured = configure_json(f)
name = input("Name: ")
if not name:
    name = "Boku No Kokoro No Y - Capitulo " + jsonConfigured["chapter"]
mangaMaker = mangaPdf(name, [requests.get(image).content for image in jsonConfigured["images"]], "./").toPdf()

URL: https://plotnf.com/reader/bokuyaba/chapter-153.00/
Name: 
PDF saved to ./Boku No Kokoro No Y - Capitulo 153.pdf
